In [ ]:
!pip install nibabel scikit-learn scipy torchvision torchio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import nibabel as nib
from scipy.ndimage import zoom
import torch
from torch.utils.data import Dataset, DataLoader, random_split

class MRIDataset(Dataset):
    def __init__(self, root_dir, target_shape=(32, 112, 112), max_per_class=400):
        self.file_paths = []
        self.labels = []
        self.target_shape = target_shape

        class_folders = ['CN', 'MCI', 'AD']
        for label, class_name in enumerate(class_folders):
            class_path = os.path.join(root_dir, class_name)
            if not os.path.exists(class_path):
                continue

            nii_files = [f for f in os.listdir(class_path) if f.endswith('.nii') or f.endswith('.nii.gz')]
            nii_files = nii_files[:max_per_class]  # sınıf başına maksimum 400 dosya
            for file in nii_files:
                full_path = os.path.join(class_path, file)
                self.file_paths.append(full_path)
                self.labels.append(label)

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        path = self.file_paths[idx]
        label = self.labels[idx]

        img = nib.load(path).get_fdata()
        img = (img - np.mean(img)) / np.std(img)

        zoom_factors = [t / s for t, s in zip(self.target_shape, img.shape)]
        img = zoom(img, zoom_factors, order=1)
        img = np.expand_dims(img, axis=0)  # [1, D, H, W]

        return torch.tensor(img, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Dataset yolu
data_dir = '/content/drive/MyDrive/ADNI_No_Preprococess'
dataset = MRIDataset(data_dir, max_per_class=400)

# Veri kümesi kontrolü
print(f"Toplam örnek sayısı: {len(dataset)}")
if len(dataset) > 0:
    print("✅ Dosyalar bulundu. İlk 5 dosya yolu:")
    for i in range(min(5, len(dataset))):
        print(dataset.file_paths[i])
else:
    raise ValueError("❌ Hiç dosya bulunamadı. Lütfen klasör yapısını ve yolunu kontrol et.")


Toplam örnek sayısı: 1200
✅ Dosyalar bulundu. İlk 5 dosya yolu:
/content/drive/MyDrive/ADNI_No_Preprococess/CN/ADNI_057_S_0934_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20100108112700337_S75850_I162801_CN.nii
/content/drive/MyDrive/ADNI_No_Preprococess/CN/ADNI_057_S_0934_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20061229160134007_S19971_I34734_CN.nii
/content/drive/MyDrive/ADNI_No_Preprococess/CN/ADNI_057_S_0934_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20081104083629388_S57842_I125126_CN.nii
/content/drive/MyDrive/ADNI_No_Preprococess/CN/ADNI_057_S_0934_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_Br_20071101174852169_S41696_I80243_CN.nii
/content/drive/MyDrive/ADNI_No_Preprococess/CN/ADNI_057_S_0934_MR_MPR-R__GradWarp__B1_Correction__N3__Scaled_Br_20070730184904436_S30605_I62965_CN.nii


In [ ]:
import os
import numpy as np
import nibabel as nib
from scipy.ndimage import zoom
import torch
from torch.utils.data import Dataset, DataLoader, random_split, Subset
import random

In [ ]:

# Dataset bölme
total_size = len(dataset)
train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size

train_ds, val_ds, test_ds = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)
test_loader = DataLoader(test_ds, batch_size=32)

print(f"\n🔍 Dataset Bölümü:")
print(f"Train: {len(train_ds)} | Val: {len(val_ds)} | Test: {len(test_ds)}")



🔍 Dataset Bölümü:
Train: 960 | Val: 120 | Test: 120


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

# 3D DenseLayer: DenseNet'in yapı taşı
class _DenseLayer3D(nn.Module):
    def __init__(self, in_channels, growth_rate, bn_size=4, drop_rate=0):
        super().__init__()
        self.norm1 = nn.BatchNorm3d(in_channels)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv3d(in_channels, bn_size * growth_rate, kernel_size=1, stride=1, bias=False)

        self.norm2 = nn.BatchNorm3d(bn_size * growth_rate)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv3d(bn_size * growth_rate, growth_rate, kernel_size=3, stride=1, padding=1, bias=False)

        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = self.conv1(self.relu1(self.norm1(x)))
        new_features = self.conv2(self.relu2(self.norm2(new_features)))
        if self.drop_rate > 0:
            new_features = F.dropout3d(new_features, p=self.drop_rate, training=self.training)
        return torch.cat([x, new_features], 1)


# 3D DenseBlock
class _DenseBlock3D(nn.Module):
    def __init__(self, num_layers, in_channels, growth_rate, bn_size=4, drop_rate=0):
        super().__init__()
        layers = []
        for i in range(num_layers):
            layers.append(_DenseLayer3D(in_channels + i * growth_rate, growth_rate, bn_size, drop_rate))
        self.block = nn.Sequential(*layers)

    def forward(self, x):
        return self.block(x)


# 3D Transition layer
class _Transition3D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.norm = nn.BatchNorm3d(in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv = nn.Conv3d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.pool = nn.AvgPool3d(kernel_size=2, stride=2)

    def forward(self, x):
        x = self.conv(self.relu(self.norm(x)))
        x = self.pool(x)
        return x


# 3D DenseNet modeli
class DenseNet3D(nn.Module):
    def __init__(self, num_classes=3, growth_rate=32, block_config=(6, 12, 24, 16),
                 num_init_features=64, bn_size=4, drop_rate=0):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv3d(1, num_init_features, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm3d(num_init_features),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        )

        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock3D(num_layers, num_features, growth_rate, bn_size, drop_rate)
            self.features.add_module(f'denseblock{i+1}', block)
            num_features += num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = _Transition3D(num_features, num_features // 2)
                self.features.add_module(f'transition{i+1}', trans)
                num_features = num_features // 2

        self.features.add_module('norm_final', nn.BatchNorm3d(num_features))

        self.classifier = nn.Sequential(
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool3d((1, 1, 1)),
            nn.Flatten(),
            nn.Linear(num_features, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(256, num_classes)
        )

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm3d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
                init.constant_(m.bias, 0)



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DenseNet3D(num_classes=3).to(device)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

def evaluate_model(loader, name):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            correct += (preds == labels).sum().item()
            total += labels.size(0)
            total_loss += loss.item()

    avg_loss = total_loss / len(loader)
    acc = correct / total * 100
    print(f"{name} | Accuracy: {acc:.2f}% | Loss: {avg_loss:.4f}")

    # Ek metrikler
    print("\n📊 Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=["CN", "MCI", "AD"], digits=4))

    print("🧩 Confusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))

    return avg_loss, acc

# Eğitim parametreleri
num_epochs = 7
patience = 3
best_val_loss = float('inf')
epochs_without_improvement = 0

print(f"\nTrain: {len(train_ds)} | Val: {len(val_ds)} | Test: {len(test_ds)}")

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    print(f"\nEpoch {epoch+1}/{num_epochs}")

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        if (batch_idx + 1) % 2 == 0 or (batch_idx + 1) == len(train_loader):
            acc = correct / total * 100
            print(f"  Batch {batch_idx+1} | Loss: {loss.item():.4f} | Accuracy: {acc:.2f}%")

    avg_train_loss = total_loss / len(train_loader)
    train_acc = correct / total * 100
    print(f"Epoch {epoch+1} Sonu | Train Loss: {avg_train_loss:.4f} | Train Accuracy: {train_acc:.2f}%")

    # Doğrulama
    val_loss, val_acc = evaluate_model(val_loader, f"Validation (Epoch {epoch+1})")

    # Erken durdurma kontrolü
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
        print(f"  ⚠️ Validation loss iyileşmedi. ({epochs_without_improvement}/{patience})")
        if epochs_without_improvement >= patience:
            print(f"  🛑 Validation loss {patience} epoch boyunca iyileşmedi. Eğitim durduruluyor.")
            break

# Final test
print("\n=== Final Test Evaluation ===")
test_loss, test_acc = evaluate_model(test_loader, "Test Set")


Train: 960 | Val: 120 | Test: 120

Epoch 1/7
  Batch 2 | Loss: 1.0755 | Accuracy: 39.06%
  Batch 4 | Loss: 1.0801 | Accuracy: 38.28%
  Batch 6 | Loss: 1.1221 | Accuracy: 38.02%
  Batch 8 | Loss: 1.1488 | Accuracy: 39.45%
  Batch 10 | Loss: 0.9860 | Accuracy: 41.88%
  Batch 12 | Loss: 1.0865 | Accuracy: 41.41%
  Batch 14 | Loss: 1.0488 | Accuracy: 40.85%
  Batch 16 | Loss: 1.0432 | Accuracy: 41.60%
  Batch 18 | Loss: 1.1049 | Accuracy: 41.67%
  Batch 20 | Loss: 0.9879 | Accuracy: 42.81%
  Batch 22 | Loss: 1.1021 | Accuracy: 42.76%
  Batch 24 | Loss: 1.0125 | Accuracy: 43.36%
  Batch 26 | Loss: 0.9313 | Accuracy: 44.59%
  Batch 28 | Loss: 1.0366 | Accuracy: 44.75%
  Batch 30 | Loss: 1.0032 | Accuracy: 45.31%
Epoch 1 Sonu | Train Loss: 1.0568 | Train Accuracy: 45.31%
Validation (Epoch 1) | Accuracy: 28.33% | Loss: 1.4420

📊 Classification Report:
              precision    recall  f1-score   support

          CN     0.2833    1.0000    0.4416        34
         MCI     0.0000    0.0000 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


  Batch 2 | Loss: 0.7914 | Accuracy: 82.81%
  Batch 4 | Loss: 0.7816 | Accuracy: 81.25%
  Batch 6 | Loss: 0.7983 | Accuracy: 77.08%
  Batch 8 | Loss: 0.7301 | Accuracy: 76.95%
  Batch 10 | Loss: 0.9213 | Accuracy: 75.94%
  Batch 12 | Loss: 0.7900 | Accuracy: 75.00%
  Batch 14 | Loss: 0.7462 | Accuracy: 74.55%
  Batch 16 | Loss: 0.7827 | Accuracy: 74.22%
  Batch 18 | Loss: 0.9229 | Accuracy: 73.26%
  Batch 20 | Loss: 0.7314 | Accuracy: 73.75%
  Batch 22 | Loss: 0.7725 | Accuracy: 73.72%
  Batch 24 | Loss: 0.8404 | Accuracy: 73.57%
  Batch 26 | Loss: 0.8669 | Accuracy: 73.20%
  Batch 28 | Loss: 0.7265 | Accuracy: 73.10%
  Batch 30 | Loss: 0.7195 | Accuracy: 73.44%
Epoch 2 Sonu | Train Loss: 0.8079 | Train Accuracy: 73.44%
Validation (Epoch 2) | Accuracy: 32.50% | Loss: 1.1313

📊 Classification Report:
              precision    recall  f1-score   support

          CN     0.3009    1.0000    0.4626        34
         MCI     0.0000    0.0000    0.0000        39
          AD     0.7143   

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


  Batch 2 | Loss: 0.6784 | Accuracy: 84.38%
  Batch 4 | Loss: 0.6240 | Accuracy: 87.50%
  Batch 6 | Loss: 0.6741 | Accuracy: 87.50%
  Batch 8 | Loss: 0.6133 | Accuracy: 85.94%
  Batch 10 | Loss: 0.5463 | Accuracy: 86.88%
  Batch 12 | Loss: 0.5843 | Accuracy: 87.24%
  Batch 14 | Loss: 0.6647 | Accuracy: 86.83%
  Batch 16 | Loss: 0.5731 | Accuracy: 87.11%
  Batch 18 | Loss: 0.5598 | Accuracy: 86.46%
  Batch 20 | Loss: 0.6641 | Accuracy: 86.41%
  Batch 22 | Loss: 0.5553 | Accuracy: 86.36%
  Batch 24 | Loss: 0.5057 | Accuracy: 86.33%
  Batch 26 | Loss: 0.5851 | Accuracy: 86.30%
  Batch 28 | Loss: 0.5814 | Accuracy: 86.16%
  Batch 30 | Loss: 0.4984 | Accuracy: 86.46%
Epoch 3 Sonu | Train Loss: 0.5936 | Train Accuracy: 86.46%
Validation (Epoch 3) | Accuracy: 53.33% | Loss: 0.9634

📊 Classification Report:
              precision    recall  f1-score   support

          CN     0.6286    0.6471    0.6377        34
         MCI     0.4521    0.8462    0.5893        39
          AD     0.7500   